# 2D model without preprocessing

In [ ]:
# 3rd-party imports
import numpy as np
import nibabel as nib
import tensorflow as tf
import tensorflow.keras as k
import matplotlib.pyplot as plt

## Get dataset & split train/test

In [ ]:
from utils.load_data import get_dataset, NibDataSequence

train_nib, val_nib = get_dataset('../data', verbose=True)

In [ ]:
train_nib.keys()

In [ ]:
[len(v) for v in train_nib.values()]

In [ ]:
[len(v) for v in val_nib.values()]

## Load train & analyze

In [ ]:
train_seq = NibDataSequence(train_nib)
len(train_seq)

In [ ]:
train_x, train_y = train_seq.load_all()

In [ ]:
train_x[0].shape, train_y[0].shape

In [ ]:
train_x[10].shape, train_y[10].shape

In [ ]:
i_data = 0
i_slice = 20

plt.figure(figsize=(10, 10))
plt.subplot(1, 3, 1)
plt.imshow(train_x[i_data][:,:,i_slice,0])

plt.subplot(1, 3, 2)
plt.imshow(train_x[i_data][:,:,i_slice,1])

plt.subplot(1, 3, 3)
plt.imshow(train_y[i_data][:,:,i_slice])

## Prepare the model